In [33]:
from sklearn import model_selection
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pylab import *
import math
%matplotlib inline 
%config InlineBackend.figure_format="retina" 

In [34]:
# 数据处理
continus={}
def ReadData(pathname):
    data=pd.read_csv(pathname)
    data.loc[data['好瓜']=='是', '好瓜'] = 1
    data.loc[data['好瓜']=='否', '好瓜'] = 0
    
    
    data = data.iloc[:,1:]
    
    
    attribute=data.columns
    for i in attribute[:-1]:
        m=0
        if(i!="密度" and i!="含糖率"):
            for j in data[i].unique():
                data.loc[data[i]==j, i]=m
                m+=1
        else:
            for j in data[i].unique():
                continus[i,str(j)]=m
                m+=1
            
    
    label=data['好瓜'].unique()
    
    #print(data)
    #print(continus)
     
    return data,attribute,label

In [35]:
def calContProb(data,feature):
    if(feature=="密度" or feature=="含糖率"):
        return calContProbCon(data,feature)
    else:
        return calContProbDis(data,feature)
        
        
def calContProbCon(data,feature):
    
    prob=np.zeros((len(data['好瓜'].unique()),len(data[feature].unique())))
    
    for i in data['好瓜'].unique():
        for j in data[feature].unique():
            mean=data[data['好瓜']==i][feature].mean()
            std=data[data['好瓜']==i][feature].std()
            
            pos=continus[feature,str(j)]
            prob[i,pos]=exp(-((j-mean)*(j-mean))/(2*std*std))/(sqrt(2*math.pi)*std)
            prob[i,pos]=round(prob[i,pos],3)#保留三位小数
            
            #print(j,prob[i,pos])
            
    
    return prob
    
def calContProbDis(data,feature):
    
    ni=len(data[feature].unique())
    prob=np.zeros((len(data['好瓜'].unique()),len(data[feature].unique())))
    
    for i in data['好瓜'].unique():
        nc=len(data[data['好瓜']==i])+ni
        for j in data[feature].unique():
            #使用拉普拉斯修正
            prob[i,j]=(len(data[data['好瓜']==i][data[feature]==j])+1)/nc
            
            #不使用拉普拉斯修正
            #prob[i,j]=(len(data[data['好瓜']==i][data[feature]==j]))/(nc-ni)
            
            prob[i,j]=round(prob[i,j],3)#保留三位小数
            
    return prob

In [36]:
def trainBayes(data,attribute):
    prob={}
    for i in attribute:
        if i!='好瓜':
            prob[i]=calContProb(pd.DataFrame(data, columns=['好瓜', i]), i)
    return prob

In [46]:
def infer(model,label,attribute,labelData):
    res = {}
    reslog={}
    for i in labels:
        #使用拉普拉斯修正
        res[i] = (len(data[data['好瓜']==i])+1)/(len(data)+2) #N=2
        reslog[i]=log((len(data[data['好瓜']==i])+1)/(len(data)+2))
        
        #不使用拉普拉斯修正
        #res[i] = (len(data[data['好瓜']==i]))/(len(data)) #N=2
        #reslog[i]=log((len(data[data['好瓜']==i]))/(len(data)))
        
        #if i==0:
        #    print(res[i])
        
        for j in attribute:

            res[i]*=model[j][i][0]
            reslog[i] += log(model[j][i][0])
            print(j,i,model[j][i][0],res[i])

    return res

In [47]:
def judge(res):
    if(res[1]>=res[0]):
        print('判断为好瓜')
    else:
        print('判断为坏瓜')

In [48]:
if __name__ == '__main__':
    data,attribute,labels=ReadData('../../data/watermelon_3.csv')
    print(attribute)
    model = trainBayes(data,attribute)
    #print(model)
    result = infer(model, labels, attribute[:-1],data['好瓜'])
    judge(result)
    print(result)
    

Index(['色泽', '根蒂', '敲声', '纹理', '脐部', '触感', '密度', '含糖率', '好瓜'], dtype='object')
色泽 1 0.375 0.1764705882352941
根蒂 1 0.625 0.11029411764705882
敲声 1 0.75 0.08272058823529412
纹理 1 0.875 0.07238051470588235
脐部 1 0.625 0.04523782169117647
触感 1 0.75 0.03392836626838235
密度 1 1.959 0.06646566951976103
含糖率 1 0.788 0.05237494758157169
色泽 0 0.333 0.17629411764705882
根蒂 0 0.333 0.05870594117647059
敲声 0 0.444 0.026065437882352945
纹理 0 0.222 0.005786527209882353
脐部 0 0.222 0.0012846090405938826
触感 0 0.667 0.0008568342300761197
密度 0 1.203 0.0010307715787815721
含糖率 0 0.066 6.803092419958376e-05
判断为好瓜
{1: 0.05237494758157169, 0: 6.803092419958376e-05}


/Users/wangxu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:38: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
